In [142]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

In [143]:
model = load_model('./conv_test.model')

In [144]:
def cloth_labels(num):
    if num == 0:
        return("T-shirt/top")
    elif num == 1:
        return('Trouser')
    elif num == 2:
        return('Pullover')
    elif num == 3:
        return('Dress')
    elif num == 4:
        return('Coat')
    elif num == 5:
        return('Sandal')
    elif num == 6:
        return('Shirt')
    elif num == 7:
        return('Sneaker')
    elif num == 8:
        return('Bag')
    elif num == 9:
        return('Ankle boot')
    else:
        return('Wrong label')

In [145]:
def detect_body(part):
    labels_counter = 0
    
    for x,y,w,h in part:

        upper_img = gray[y:y+w,x:x+w]
        resized = cv2.resize(upper_img,(28,28))
        normalized = resized/255.0
        reshaped = np.reshape(normalized,(1,28,28,1))
        result = model.predict(reshaped)

        labels_counter = np.argmax(result,axis=1)[0]
        labels = np.argsort(result,axis=1)[0]
        
        
        cloth = cloth_labels(labels_counter)

        cv2.rectangle(resize_img,(x,y),(x+w,y+h),(0,0,255),2)
#         cv2.putText(
#           resize_img, cloth, 
#           (x, y+5),
#           cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,255),2)
    
    return labels_counter


In [146]:
def print_labels(arr):
    '''
    Prints top 3 most classfied items.
    '''
    arr = np.multiply(arr, -1)
    label_sorted = np.argsort(arr)
    print("Top 3 items")
    for x in label_sorted[:3]:
        print(cloth_labels(x))

In [147]:
def print_all_labels(arr):
    '''
    Prints all labels that classified
    '''
    arr = np.multiply(arr, -1)
    label_sorted = np.argsort(arr)
    for x in label_sorted:
        if arr[x]!=0:
            print(cloth_labels(x))

In [148]:
source = cv2.VideoCapture('./example6.mp4')

# source=cv2.VideoCapture(0)


upper_body = cv2.CascadeClassifier('/Users/katyang/opt/miniconda3/envs/2812ICT_env/lib/python3.6/site-packages/cv2/data/haarcascade_upperbody.xml')
lower_body = cv2.CascadeClassifier('/Users/katyang/opt/miniconda3/envs/2812ICT_env/lib/python3.6/site-packages/cv2/data/haarcascade_lowerbody.xml')
full_body = cv2.CascadeClassifier('/Users/katyang/opt/miniconda3/envs/2812ICT_env/lib/python3.6/site-packages/cv2/data/haarcascade_fullbody.xml')

labels_counter = [0,0,0,0,0,0,0,0,0,0]

while(True):

    ret,img = source.read()
    resize_img = cv2.resize(img, (0, 0), fx=0.2, fy=0.2)
    gray = cv2.cvtColor(resize_img,cv2.COLOR_BGR2GRAY)
    
    upper = upper_body.detectMultiScale(gray,1.07,4)  
    lower = lower_body.detectMultiScale(gray,1.04,3)  
    full = full_body.detectMultiScale(gray,1.05,4)  
    
    label1 = detect_body(upper)
    label2 = detect_body(lower)
    label3 = detect_body(full)
    
    labels_counter[label1] += 1
    labels_counter[label2] += 1
    labels_counter[label3] += 1
    
    reverse = np.multiply(labels_counter, -1)
    sorted_label = np.argsort(reverse)
    
    cv2.putText(resize_img, 
                cloth_labels(sorted_label[0]), 
                (0, 15),
                cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,255),2)
    
    cv2.putText(resize_img, 
                cloth_labels(sorted_label[1]), 
                (0, 30),
                cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,255),2)
    
    cv2.putText(resize_img, 
                cloth_labels(sorted_label[2]), 
                (0, 45),
                cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,255),2)
    
    cv2.imshow('LIVE',resize_img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break

# Release capture
source.release()
cv2.destroyAllWindows()

labels_counter
print_labels(labels_counter)

Top 3 items
T-shirt/top
Bag
Trouser


In [141]:
print_all_labels(labels_counter)

Shirt
T-shirt/top
Bag
Ankle boot
Trouser
Dress
Coat
Sandal
